In [97]:
import numpy as np
import tensorflow as tf

In [1]:
from tqdm import tqdm

import transformers
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoConfig
from transformers import BigBirdTokenizer, BigBirdForQuestionAnswering
import torch

import gc
import pandas as pd

2021-11-20 09:26:54.409317: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [ ]:
#'/home/ubuntu/kylee/Backup/output_backup/klue-mrc/version_5/transformers',

model = BigBirdForQuestionAnswering.from_pretrained(
    '/home/ubuntu/2021AiHub-ODQA/models/korquad_2/0-2-ckpt',
)
tokenizer = AutoTokenizer.from_pretrained(
    '/home/ubuntu/2021AiHub-ODQA/models/korquad_2/0-2-ckpt', 
    use_fast=False
)

In [39]:
x_valid = pd.read_csv('./models/x_valid_light.csv')
x_valid

,title,text,question,answer,answer_start,answer_end
0,현대 니트패션에 나타난 상.하위문화의 혼합특성,스타일이 연출되었는데 상?하위의 개념에서도 전통적인 개념의 경계를 무너뜨리는 혼합...,상류층의 의상과 대중들의 의상의 경계가 확실했던 대립관계의 경계가 해체되기 시작한 ...,1960년대의 영 패션이 등장,188,204
1,이공계열 학생들의 탐구능력 향상과 자기주도적 학습을 돕는 학생연구프로그램의 운영 성...,"설문 조사와 심층 인터뷰, 평가회 등을 통하여 학생연구프로그램의 참여를 통하여 습...",자기주도 학습이란?,"학습자가 주도권을 가지고, 타인의 도움을 받아서, 혹은 그러한 도움 없이, 자신의 ...",241,378
2,패션제품 광고의 색채 배색에 대한 광고 회상 연구,광고에서의 색채연구는 컬러마케팅의 관점에서 색상이 브랜드 이미지를 나타내는 도구로...,단색 배색은 무엇인가?,한 가지 색의 조화로 한 가지 색을 다양한 명도와 채도로 조화시킨 배색,180,219
3,ISO 15016 해석법에 의한 사고선박 예인 시 파랑 중 부가저항 추정 프로그램 개발,appropriate to use for predicting the resista...,사고선박의 저항은 어떻게 계산하는가?,사고선박 자체의 정수중 선체 저항에 각종 외력에 대한 부가저항을 더하여 계산,254,296
4,손 관련 인체측정자료를 이용한 한국인의 손 모양 유형 분류 및 특성 분석,장갑과 기기를 조작하는데 필요한 조종 장치를 들 수 있다. 치수 및 형태가 적합한 ...,치수 및 형태가 적합한 장갑의 착용은 보호 장구로서의 역할을 하지만 부적절하게 설계...,손의 감각 및 악력이 저하되는 문제점,132,152
...,...,...,...,...,...,...
9995,대안적 인지 이론으로서 '자원 기반 관점'에 대한 이론적 고찰과 시험 적용,"것은 대체되어야 할 학습의 장애물이 아니라, 교사가 수업 중에 과학적인 아이디어를 ...",RBV에 따르면 오개념은 어떤 기반이 되는가?,"대체되어야 할 학습의 장애물이 아니라, 교사가 수업 중에 과학적인 아이디어를 전개하...",3,72
9996,실제 도로 주행과정에서 타이어와 도로의 마찰에 의해서 발생하는 미세입자의 특성연구,μg/m3까지 증가하게 된다. 이는 차량 속도의 증가에 따라서 마모되는 타이어 입...,차량 속도가 증가함에 따라서 2.5 μm 부근에서 중량 농도값이 피크값을 나타낸 이유는?,차량 속도의 증가에 따라서 마모되는 타이어 입자가 증가하고 도로면에 존재하는 미세먼...,21,105
9997,족양명위경 하지부 유주에 관한 고찰 - &#12302;중국침구경락통감&#12303;...,재 中趾가 아닌 足二趾 말단外側에서 취혈3)하고 있어 족양명위경의 경로와 정확하게 ...,中國鍼灸經絡通鑑이란 무엇인가?,"중국침구학회 경락연구회와 중국중의연구원 침구연구소를 중심으로 鄧良月, 黃龍祥 등 중...",209,426
9998,지반굴착에 따른 지반 안정성 평가 시 지하시설물이 지하수흐름에 미치는 영향 분석,"사항인 지하수유동 특성은 굴착구간 주변의 존재하는 지하시설물의 규모(size), 이...",지반굴착에 따른 지반 안정성 평가 시 고려사항인 지하수유동 특성은 어떤 영향을 받나요?,"굴착구간 주변의 존재하는 지하시설물의 규모(size), 이격거리(separation...",14,130


In [71]:
sample = x_valid[9985:9990]
display(list(sample.question), sample.text)

['불투수성포장의 부정적 영향은?',
 '무는 인체에서 어떤 작용을 하는가?',
 '이산화탄소 지중저장이란?',
 '부모역할의 주요 하위개념은 무엇인가?',
 '조직 내에서 이직의 증가는 어떤 영향을 미치는가?']

9985    심광장의 경우 자연순환기능이 상실되면서 지표면 온도가 상승되는 원인을 제공하고 있으...
9986    에서 쉽게 섭취할 수 있는 재료인 당근은 비타민A의 전구체로 카로틴을 많이 함유하고...
9987    라이트;고온.고압;지화학 반응;용해; carbon dioxide;zeolite;hi...
9988     현대사회는 진로와 직업, 지역사회활동, 정치적 참여, 예술, 취미, 스포츠 등과 ...
9989    ehavior;Turnover intention; 본문 최근 의료 환경의 급속한 변...
Name: text, dtype: object

In [72]:
encodings = tokenizer(list(sample.question)[0], 
                      list(sample.text)[0],
                      return_tensors="pt",
                      padding=True, truncation=True)

In [73]:
# input 을 cuda로 저장
print("type ",type(encodings))
print(encodings)

# input_ids = encodings["input_ids"].to(self.device).cpu()
# token_type_ids = encodings["token_type_ids"].to(self.device).cpu()
# attention_mask = encodings["attention_mask"].to(self.device).cpu()
input_ids = encodings["input_ids"]
token_type_ids = encodings["token_type_ids"]
attention_mask = encodings["attention_mask"]
# input_ids = encodings["input_ids"]
# token_type_ids = encodings["token_type_ids"]
# attention_mask = encodings["attention_mask"]

outputs = model(input_ids=input_ids,attention_mask=attention_mask)

Attention type 'block_sparse' is not possible if sequence_length: 635 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3.Changing attention type to 'original_full'...


type  <class 'transformers.tokenization_utils_base.BatchEncoding'>
{'input_ids': tensor([[    2,  3311, 23591,  4602,  4651,  4794,  4561,  7733,  4809,  7136,
          4839,   535,     3,  3512, 27680,  4561,  6822,  7124, 17669, 22250,
          4544, 10090,  5189,  4685,  4593,  9731,  4685,  9937,  4771,  7357,
          5189,  4741,  7703,  4604,  7085, 23953,  3738,  7022,  6929, 11358,
          4561,  6936,  4584,  4741,  7010,  4837,  9326, 10707,  7131, 23953,
          3738,  4742,   518,  3729,  4741,  9326,  9627,  7412,  5189,  4583,
          7308,  3693,  4521,  4935,  4544,  2554,  4625,  3738,  5243, 17391,
          4561,  8514,  9304,  4756,  6834,  7124, 17669, 22250,  4544, 12059,
          5189,  4685,  4593,  9731,  4685, 11159, 26573,  4584,  7136,  4604,
         14816,  4741,  2579,  9627,  7690,  4742,   518,  6953,   516,  3311,
         23591,  4602,  4651,  4794,  4839,  9731,  4685, 11159, 26573,  3375,
          4830,  6799,  4744,  8510,  4809,  7214,

In [74]:
start_logits, end_logits = outputs.start_logits, outputs.end_logits
token_start_index, token_end_index = start_logits.argmax(dim=-1), end_logits.argmax(dim=-1)
pred = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(encodings["input_ids"][0][token_start_index: token_end_index + 1]))
pred = pred.replace("#","")

In [76]:
token_end_index

tensor([0])

In [34]:
list(sample.text)[0]

'심광장의 경우 자연순환기능이 상실되면서 지표면 온도가 상승되는 원인을 제공하고 있으며 국내 대다수의 도시에는 이러한 광장들이 존재하고 있다. 이는 광장으로 조성되기 이전 원지반이 갖고 있던 녹지의 면적 축소와 함께 자연순환기능이 저하되면서 지표면 기온상승에 영향을 끼치는 것으로 나타났다. 또한, 불투수성포장은 지표면 기온상승 뿐만 아니라 생태적 기능을 저하시키는 결과를 초래하여 생물다양성 감소의 원인으로 작용하고 있으며 자연녹지보다 불투수성포장면적이 넓은 도시지역으로 갈수록 생태적건전성1)은 악화되고 있다. 강릉시 교동 택지 내 불투수성 포장광장의 경우도 조성 된지 16년이 지난 현재 자연순환 기능을 상실하여 도심열섬화를 가중시키는 공간으로 방치되고 있으며, 낙후된 시설들이 시민들의 안전을 위협하고 있어 그에 따른 개선 방안 수립이 절실한 상황이다. 이러한 현실에서 도시민들의 자연환경에 대한 관심은 점차 늘어나고 있는 추세이며, Yun(2016)은 도시의 생태적건전성이 도시민 삶의 질과 직결되어 있으므로 현재 그리고 미래의 세대를 위해 반드시 확보되어야 한다고 했다. 도시의 생태적건전성에 대한 문제는 국제적으로 2006년 독일에서 열린 생물다양성협약(CBD)에서 제기되어 URBIO (Urban Biodiversity and Design)가 개최되면서 본격화되었다. 우리나라에서도 2012년 도시녹지의 생태적 기능 연구가 환경부에서 진행되었고 이후로 국립환경과학원의 2013년 녹색도시 건강성 평가지표 연구 및 2014년 도시생태계 건전성과 관련된 연구가 진행된바 있다. 최근 국립환경과학원을 비롯하여 도시생태적건전성지표에 관한 다양한 연구도 활발히 진행 중에 있다. 이에 본 연구에서는 강릉시 교동 낙후된 광장의 생태적 기능을 회복 할 수 있도록 기반환경과 서식지를 복원하여 생태적건전성 확보를 통한 도심생태휴식공간으로 재탄생시키기 위한 방안을 제시하고자 한다. 또한 재원 확보의 방안으로 환경부 생태계보전협력금을 활용하여 반환사업으로 추진하는 방법을 검토하여본 연구가

In [114]:
sm = torch.nn.Softmax(dim=0)
float(torch.max(sm(outputs[0][0])))

0.7967382669448853

In [85]:
round(float(torch.max(outputs.start_logits[4])), 3)

-3.392

In [39]:
outputs.start_logits[4].argmax(dim=-1), outputs.end_logits[4].argmax(dim=-1)

(tensor(0), tensor(644))

In [77]:
arr = [ { 'answer_start': 20,
    'answer_end': 20,
    'answer': '혼잣말',
    'answer_score': [ 1.091, 1.322 ] },
      { 'answer_start': 20,
    'answer_end': 20,
    'answer': '혼잣말',
    'answer_score': [ 1.091, 1.322 ] }]
arr

[{'answer_start': 20,
  'answer_end': 20,
  'answer': '혼잣말',
  'answer_score': [1.091, 1.322]},
 {'answer_start': 20,
  'answer_end': 20,
  'answer': '혼잣말',
  'answer_score': [1.091, 1.322]}]

In [83]:
def softmax(n):
    exp_n = np.exp(n)
    sum_exp_n = np.sum(exp_n)
    return exp_n / sum_exp_n

In [102]:
max(softmax(detached))

0.7967382

In [62]:
res2 = json.dumps(tmp, ensure_ascii=False)

In [57]:
import json

res = json.dumps([
    {
    'context': 'mple of a', 
    'question': '경술국치가 뭐야?', 
    'answer': [
        {'answer_start': 20, 
            'answer_end': 24, 
            'answer': '자료수집선지식', 
            'answer_score': [2.547, 1.088]}
    ]},
    {
    'context': 'mple of a', 
    'question': '경술국치가 뭐야?', 
    'answer': [
        {'answer_start': 20, 
            'answer_end': 24, 
            'answer': '자료수집선지식', 
            'answer_score': [2.547, 1.088]}
    ]}
],ensure_ascii=False)

In [58]:
res

'[{"context": "mple of a", "question": "경술국치가 뭐야?", "answer": [{"answer_start": 20, "answer_end": 24, "answer": "자료수집선지식", "answer_score": [2.547, 1.088]}]}, {"context": "mple of a", "question": "경술국치가 뭐야?", "answer": [{"answer_start": 20, "answer_end": 24, "answer": "자료수집선지식", "answer_score": [2.547, 1.088]}]}]'

In [35]:
pred

'[CLS] 불투수성포장의 부정적 영향은 ? [SEP] 심광장의 경우 자연순환기능이 상실되면서 지표면 온도가 상승되는 원인을 제공하고 있으며 국내 대다수의 도시에는 이러한 광장들이 존재하고 있다 . 이는 광장으로 조성되기 이전 원지반이 갖고 있던 녹지의 면적 축소와 함께 자연순환기능이 저하되면서 지표면 기온상승에 영향을 끼치는 것으로 나타났다 . 또한 , 불투수성포장은 지표면 기온상승 뿐만 아니라 생태적 기능을 저하시키는 결과를 초래하여 생물다양성 감소의 원인으로 작용하고 있으며 자연녹지보다 불투수성포장면적이 넓은 도시지역으로 갈수록 생태적건전성1 ) 은 악화되고 있다 . 강릉시 교동 택지 내 불투수성 포장광장의 경우도 조성 된지 16년이 지난 현재 자연순환 기능을 상실하여 도심열섬화를 가중시키는 공간으로 방치되고 있으며 , 낙후된 시설들이 시민들의 안전을 위협하고 있어 그에 따른 개선 방안 수립이 절실한 상황이다 . 이러한 현실에서 도시민들의 자연환경에 대한 관심은 점차 늘어나고 있는 추세이며 , Yun ( 2016 ) 은 도시의 생태적건전성이 도시민 삶의 질과 직결되어 있으므로 현재 그리고 미래의 세대를 위해 반드시 확보되어야 한다고 했다 . 도시의 생태적건전성에 대한 문제는 국제적으로 2006년 독일에서 열린 생물다양성협약 ( CBD ) 에서 제기되어 URBIO ( Urban Biodiversity and Design ) 가 개최되면서 본격화되었다 . 우리나라에서도 2012년 도시녹지의 생태적 기능 연구가 환경부에서 진행되었고 이후로 국립환경과학원의 2013년 녹색도시 건강성 평가지표 연구 및 2014년 도시생태계 건전성과 관련된 연구가 진행된바 있다 . 최근 국립환경과학원을 비롯하여 도시생태적건전성지표에 관한 다양한 연구도 활발히 진행 중에 있다 . 이에 본 연구에서는 강릉시 교동 낙후된 광장의 생태적 기능을 회복 할 수 있도록 기반환경과 서식지를 복원하여 생태적건전성 확보를 통한 도심생태휴식공간으로 재탄생시키기 위한 방안을 제시하고자 한다 . 또한 재

In [17]:
import pandas as pd
chat = pd.read_csv('chatData.csv', error_bad_lines=False, quoting=3, sep=',', escapechar='\n')

/home/ubuntu/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 4866: expected 4 fields, saw 5\nSkipping line 5058: expected 4 fields, saw 5\nSkipping line 6594: expected 4 fields, saw 5\nSkipping line 49666: expected 4 fields, saw 5\nSkipping line 86850: expected 4 fields, saw 5\nSkipping line 101954: expected 4 fields, saw 5\nSkipping line 123778: expected 4 fields, saw 6\nSkipping line 129538: expected 4 fields, saw 5\n'
b'Skipping line 171458: expected 4 fields, saw 6\n'
b'Skipping line 282050: expected 4 fields, saw 5\nSkipping line 292162: expected 4 fields, saw 5\n'
b'Skipping line 432962: expected 4 fields, saw 5\nSkipping line 475714: expected 4 fields, saw 5\nSkipping line 478210: expected 4 fields, saw 5\n'
b'Skipping line 619842: expected 4 fields, saw 5\n'
b'Skipping line 741122

In [16]:
chat

,Q,A,label,embedding
0,12시 땡!,하루가 또 가네요.,0.0,"""[ 2.01795369e-01 -3.44379283e-02 1.53957367e..."
1,3.79792005e-01 1.42493680e-01 1.82864383e-...,NaN,NaN,NaN
2,-3.94889355e-01 -1.17657103e-01 -2.32989192e-...,NaN,NaN,NaN
3,-2.47104481e-01 3.14535856e-01 5.60950637e-...,NaN,NaN,NaN
4,-2.92885333e-01 -2.04060197e-01 3.35198753e-...,NaN,NaN,NaN
...,...,...,...,...
2167803,-2.34302357e-02 -1.52035654e-01 1.01029314e-...,NaN,NaN,NaN
2167804,1.15168624e-01 -5.84745221e-02 8.86170492e-...,NaN,NaN,NaN
2167805,7.53511116e-02 1.99905574e-01 3.19847763e-...,NaN,NaN,NaN
2167806,-6.96479157e-02 -1.77732170e-01 -2.13060245e-...,NaN,NaN,NaN
